In [1]:
# Identify athletes meeting 2025 Taiwan Open selection rules

#1. Singapore citizen
#2. No athlete younger than 16 years of age on 31 December 2025 may be entered or permitted to compete.
#3. Athletes may be required to compete at any identified competitions for fitness and performance monitoring at least 6 months before competition dates. These will include SA organized competitions or any other sanctioned competitions.
#4. No road races results will be accepted


%load_ext autoreload
%autoreload 2

In [2]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime
import icecream as ic

from google.cloud import storage



In [441]:
# PRODUCTION ENVIRONMENT
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, TEAM, AGE, RANK AS COMPETITION_RANK, DIVISION, EVENT, DISTANCE, EVENT_CLASS, UNIQUE_ID, DOB, NATIONALITY, WIND, CATEGORY_EVENT, GENDER, COMPETITION, DATE, YEAR, REGION
FROM `saa-analytics.results.athlete_results_prod` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='-' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL

"""

competitors = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [442]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,24/11/90,nan,,Throw,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,4/5/95,nan,,Jump,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,23/4/97,,,None,Male,Pocari Sweat Run 2.4km 2024,9/7 to 9/7,2024,Local
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,1/8/19,nan,0.5,None,Female,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,3/3/74,nan,,Throw,Male,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,28/12/02,IND,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26005,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,9/7/09,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26006,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,6/9/10,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26007,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,20/7/09,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local


In [443]:
# Create mm/dd/yy column for date of event

import datetime

#s=247.779

#datetime.datetime.fromtimestamp(s).strftime('%M:%S.%f')

for i in range(len(competitors)):
        
    rowIndex = competitors.index[i]

    date = competitors.iloc[rowIndex,16]
    year = competitors.iloc[rowIndex,17]    
    
    if 'to' in date or ' - ' in date:
        
        pos = re.search('to|\s\-\s', date)
                                                
        # Splice string to day and month

        split_pos_start=pos.start()+3

            
        final_date = date[split_pos_start:] # left string post splicing
        final_year = year[2:]
        
        event_date = final_date + '/' + final_year
        
        competitors.loc[rowIndex, 'event_date'] = event_date
        
    elif re.search('\w\-\w', date):
        
        if competitors.iloc[rowIndex, 15] == "National School Games":
        
            event_date = '04'+'/'+date[1:3] + '/' + year[2:]  # reverse order from dd/mm to mm/dd
        
            competitors.loc[rowIndex, 'event_date'] = event_date
        
        else:
            
            event_date = date + '-' + year[2:]
            
            competitors.loc[rowIndex, 'event_date'] = event_date

        
        

        
competitors['event_date'] = competitors['event_date'].astype(str)
competitors['event_date'] = competitors['event_date'].str.replace('\xa0', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\r', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\n', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.strip()
       
    

In [444]:
# Calculate number of days from today to event date

competitors['event_date_dt'] = pd.to_datetime(competitors['event_date'], errors='coerce')

competitors['delta_time']= datetime.datetime.now() - competitors['event_date_dt']

competitors['delta_time_conv'] = pd.to_numeric(competitors['delta_time'].dt.days, downcast='integer')


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_92146/2667859050.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  competitors['event_date_dt'] = pd.to_datetime(competitors['event_date'], errors='coerce')


In [445]:
# Choose last 6 months or 12 months of results 

competitors = competitors[(competitors['delta_time_conv']>=0) & (competitors['delta_time_conv']<=365)]

competitors=competitors.reset_index(drop=True)


In [446]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Athletics/')


competitors.to_csv('athletes_time_filtered.csv', sep=',', encoding='utf-8-sig', index=False)

In [447]:
competitors[competitors['NAME']=='SOH RUI YONG GUILLAUME']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv
645,SOH RUI YONG GUILLAUME,04:59.3,INDIVIDUAL,33,12,None,1 mile,None,None,None,...,Mid,Male,Vitaly West Mininster Mile 5K (UK),22-Sep,2024,International,22-Sep-24,2024-09-22,205 days 21:20:07.814386,205.0
646,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33,133,None,10000m,None,None,None,...,Long,Male,"Tokai University Long Distance Challenge 10,0...",25-Nov,2024,International,25-Nov-24,2024-11-25,141 days 21:20:07.814386,141.0
647,SOH RUI YONG GUILLAUME,15:07.7,INDIVIDUAL,33,46,None,5000m,None,None,None,...,Long,Male,Friday Night Under the Lights 5K (UK),13-Sep,2024,International,13-Sep-24,2024-09-13,214 days 21:20:07.814386,214.0
648,SOH RUI YONG GUILLAUME,08:35.8,INDIVIDUAL,33,None,None,3000m,None,None,None,...,Long,Male,Hercules Wimbledon Athletics Club 3000m,14-Jun,2024,International,14-Jun-24,2024-06-14,305 days 21:20:07.814386,305.0


In [448]:
mask = (competitors['DIVISION'].str.contains(r'B', na=False) & competitors['COMPETITION'].str.contains(r'National School Games', na=False))
competitors.loc[mask, 'AGE'] = '16'

In [449]:
competitors['DOB_new'] = pd.to_datetime(competitors['DOB'], errors='coerce')

competitors['year_extract']=competitors['DOB_new'].dt.strftime('%Y')

competitors['year_extract']=pd.to_numeric(competitors['year_extract'])

competitors['age_extract'] = 2025 - competitors['year_extract']


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_92146/3522909459.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  competitors['DOB_new'] = pd.to_datetime(competitors['DOB'], errors='coerce')


In [450]:
def age(number):  # correct negative age numbers
    
    if number<0:
        
        number+=100
        
    return number


competitors['age_extract'] = competitors['age_extract'].apply(age)


In [451]:
# If NSG event then choose AGE otherwise choose age_extract

condition1=competitors['COMPETITION']=='National School Games'


competitors['age_extract']=competitors['AGE'].where((condition1), competitors['age_extract'].values)


In [452]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,DOB_new,year_extract,age_extract
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,11/9 to 11/10,2024,Local,11/10/24,2024-11-10,156 days 21:20:07.814386,156.0,1990-11-24,1990.0,35.0
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,11/9 to 11/10,2024,Local,11/10/24,2024-11-10,156 days 21:20:07.814386,156.0,1995-04-05,1995.0,30.0
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,9/7 to 9/7,2024,Local,9/7/24,2024-09-07,220 days 21:20:07.814386,220.0,1997-04-23,1997.0,28.0
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,6/22 to 6/22,2024,Local,6/22/24,2024-06-22,297 days 21:20:07.814386,297.0,2019-01-08,2019.0,6.0
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,6/22 to 6/22,2024,Local,6/22/24,2024-06-22,297 days 21:20:07.814386,297.0,2074-03-03,2074.0,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14866,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,2/23 to 3/2,2025,Local,3/2/25,2025-03-02,44 days 21:20:07.814386,44.0,2002-12-28,2002.0,23.0
14867,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,2/23 to 3/2,2025,Local,3/2/25,2025-03-02,44 days 21:20:07.814386,44.0,2009-09-07,2009.0,16.0
14868,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,2/23 to 3/2,2025,Local,3/2/25,2025-03-02,44 days 21:20:07.814386,44.0,2010-06-09,2010.0,15.0
14869,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,2/23 to 3/2,2025,Local,3/2/25,2025-03-02,44 days 21:20:07.814386,44.0,2009-07-20,2009.0,16.0


In [453]:
competitors['AGE'] = competitors['AGE'].replace(regex=r'-', value=np.NaN)
competitors['AGE'] = competitors['AGE'].replace(regex=r'nan', value=np.NaN)


competitors['AGE'] = pd.to_numeric(competitors['AGE'])

#athletes_selected = competitors[((competitors['AGE']>=15) & (competitors['AGE']<=17))]

#athletes_selected = competitors[(competitors['YEAR']=='2024')]

In [454]:
# Converts any time format into seconds

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()
    
   # print('metric', metric)
    
    try:
        
        if 'w' in metric:  # skip marks with illegal wind speeds
            
            print('W', metric)
            
            output=''
            
        else:
            
    
            if any(s in string for s in l)==True:
            
                if 'm' in metric:
            
                    metric=metric.replace('m', '')
                    output=float(str(metric))
            
                elif 'GR' in metric:
            
                    metric=metric.replace('GR', '')
                    output=float(str(metric))
                
          #  elif 'w' in metric:
                
               
           #     metric1=metric.replace('w', '')
                
           #     output=float(str(metric1))
                
                else:
    
                    output=float(str(metric))
        
        #elif 'w' in metric:
            
                
         #       metric1=metric.replace('w', '')
                             
         #       output=float(str(metric1))
                
                
        
            else:
        
                searchstring = ":"
                searchstring2 = "."
                substring=str(metric)
                count = substring.count(searchstring)
                count2 = substring.count(searchstring2)
            
                if count==0:
                
                    output=float(substring)
            
            
      #      elif 'marathon' in string and count==2:
                
      #          print('HERE', metric)
                
      #          metric = metric.replace(":", ".", 2)
                
      #          print('METRIC', metric)
                
       #         h,m,s = metric.split(':')            

       #         output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())

               
             
                elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                
                                                
                    time=str(metric)
                    h, m ,s = time.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
                elif (count==1 and count2==1):
            
                    m,s = metric.split(':')
                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                     
                elif (count==1 and count2==2):
                
            
                    metric = metric.replace(".", ":", 1)
            
                    h,m,s = metric.split(':')            
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
                elif (count==2 and count2==0):
                
            
                    h,m,s = metric.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
  
            

    except:
        
        pass
                
    return output

In [455]:
athletes=competitors

In [456]:
for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()

# Create temporary mapped event column

athletes['MAPPED_EVENT']=''

# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'


# Running

#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'50', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '50m'
#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'60', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '60m'
#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'80', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'

mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'

mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'300 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '300m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'

mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'^600$', na=True, regex=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '600m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'800', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1000m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1500', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'3000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes['EVENT'].str.contains(r'^3000m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'5000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'^5000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'10000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'Mile', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1 Mile'

#mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'



# Hurdles



mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles|Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('0.838|0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles(0.838m)'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['REGION'].str.contains('International', na=False) & athletes['EVENT_CLASS'].str.contains('None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles(0.838m)'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['DIVISION'].str.contains('Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles(0.838m)'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'100', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles(0.838m)'



mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('1.067', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'
mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['REGION'].str.contains('International', na=False) & athletes['EVENT_CLASS'].str.contains('None', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))  # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'
mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Male', na=False) & athletes['EVENT_CLASS'].str.contains('None', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'



mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.067', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'


mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.067', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'
#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(' ', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.762m)'



mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'


mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.762m)'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762m', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.762m)'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.762m)'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False)  & athletes['EVENT_CLASS'].str.contains(r'None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.762m)'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'



mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False)  & athletes['REGION'].str.contains('International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'



# Throws


mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'600', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw(600g)'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['DIVISION'].str.contains(r'Open|OPEN', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) #NEW
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw(600g)'


mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'800', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw(800g)'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['DIVISION'].str.contains(r'Open|OPEN', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw(800g)'



mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS'].str.contains(r'4', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(4kg)'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Male') & (athletes['EVENT_CLASS'].str.contains(r'7', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(7.26kg)'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['DIVISION'].str.contains(r'Open', na=False)))# NEW
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(4kg)'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Male') & (athletes['DIVISION'].str.contains(r'Open', na=False)))# NEW
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(7.26kg)'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['REGION'].str.contains(r'International', na=False)) & (athletes['GENDER']=='Male'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(7.26kg)'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['REGION'].str.contains(r'International', na=False)) & (athletes['GENDER']=='Female'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(4kg)'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))& (athletes['GENDER']=='Female'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(4kg)'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))& (athletes['GENDER']=='Male'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(7.26kg)'



mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS'].str.contains(r'4', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw(4kg)'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False, regex=True) & (athletes['GENDER']=='Male') & (athletes['EVENT_CLASS'].str.contains(r'7', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw(7.26kg)'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))& (athletes['GENDER']=='Female'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw(4kg)'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))& (athletes['GENDER']=='Male'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw(7.26kg)'



mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw|Discus', na=False) & athletes['EVENT_CLASS'].str.contains(r'2', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw(2kg)'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw|Discus', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.00kg|1kg', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw(1kg)'

mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw|Discus', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # New
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw(2kg)'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw|Discus', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # New
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw(1kg)'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw(1kg)'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw(2kg)'



# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump|High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'

mask = athletes['EVENT'].str.contains(r'^Long\sJump$|^Long\sjump$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump|Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault|Pole vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'

# Steeplechase

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'3000m steeplechase', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase(0.762m)'
mask = (athletes['EVENT'].str.contains(r'3000m steeplechase', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase(0.914m)'

mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False)  & athletes['DISTANCE'].str.contains(r'3000', na=False) & athletes['GENDER'].str.contains(r'Female', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase(0.762m)'
mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False)  & athletes['DISTANCE'].str.contains(r'3000', na=False) & athletes['GENDER'].str.contains(r'Male', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase(0.914m)'


#mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'


# Marathon

mask = athletes['EVENT'].str.contains(r'^Marathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\sMarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\smarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'


# Walk

#mask = athletes['EVENT'].str.contains(r'1500m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'1500', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'


#mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'


#mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'5000m Race Walk', na=True) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'

#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'5000', na=False) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'

#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'10000', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Racewalk'


#mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Race Walk'

# Relay

#mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'

mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'1600', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'




In [457]:
athletes.reset_index(drop=True)


,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,DOB_new,year_extract,age_extract,MAPPED_EVENT
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34.0,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,2024,Local,11/10/24,2024-11-10,156 days 21:20:07.814386,156.0,1990-11-24,1990.0,35.0,Hammer Throw(7.26kg)
1,"Koh, Wei Shien",4.41m,NUS,29.0,3,Open,Pole Vault,0,,L435B06,...,2024,Local,11/10/24,2024-11-10,156 days 21:20:07.814386,156.0,1995-04-05,1995.0,30.0,Pole Vault
2,"Wen Qiang, Koh",07:38.0,SIT,27.0,4,Tertiary Men,Run,2400,,,...,2024,Local,9/7/24,2024-09-07,220 days 21:20:07.814386,220.0,1997-04-23,1997.0,28.0,
3,"Harinee, Sivakumaru",15.03,SKM,4.0,7,5-Jun,Run,60,,nan,...,2024,Local,6/22/24,2024-06-22,297 days 21:20:07.814386,297.0,2019-01-08,2019.0,6.0,
4,"Din, Muhamad Edir",22.43m,SMA,50.0,5,Masters,Javelin Throw,0,,nan,...,2024,Local,6/22/24,2024-06-22,297 days 21:20:07.814386,297.0,2074-03-03,2074.0,51.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14866,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23.0,2,Open,Relay,400,Trial,None,...,2025,Local,3/2/25,2025-03-02,44 days 21:20:07.814386,44.0,2002-12-28,2002.0,23.0,4 x 100m
14867,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16.0,2,Open,Relay,400,Trial,None,...,2025,Local,3/2/25,2025-03-02,44 days 21:20:07.814386,44.0,2009-09-07,2009.0,16.0,4 x 100m
14868,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15.0,2,Open,Relay,400,Trial,None,...,2025,Local,3/2/25,2025-03-02,44 days 21:20:07.814386,44.0,2010-06-09,2010.0,15.0,4 x 100m
14869,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16.0,2,Open,Relay,400,Trial,None,...,2025,Local,3/2/25,2025-03-02,44 days 21:20:07.814386,44.0,2009-07-20,2009.0,16.0,4 x 100m


In [458]:
# Convert results and seed into seconds format

for i in range(len(athletes)):
    
    result_out=''
        
    rowIndex = athletes.index[i]

    input_string=athletes.iloc[rowIndex,6]    # event description
    
    metric=athletes.iloc[rowIndex,1] # result
    
    if metric=='—' or metric=='DQ' or metric=='SCR' or metric=='FS' or metric=='DNQ' or metric=='DNS' or metric=='NH' or metric=='NM' or metric=='FOUL' or metric=='DNF' or metric=='SR' :
        continue
    
    result_out = convert_time(i, input_string, metric)
    print('line', i, input_string, metric, result_out)

    
         
    athletes.loc[rowIndex, 'RESULT_CONV'] = result_out


line 0 Hammer Throw 32.40m 32.4
line 1 Pole Vault 4.41m 4.41
line 2 Run 07:38.0 458.0
line 3 Run 15.03 15.03
line 4 Javelin Throw 22.43m 22.43
line 5 Shot Put 8.56m 8.56
line 6 Discus Throw 22.96m 22.96
line 7 Shot Put 8.14m 8.14
line 8 Run 07:00.5 420.5
line 9 Race Walk 28:09.6 1689.6
line 10 Dash 48.68 48.68
line 11 Triple Jump 6.41m 6.41
line 12 Dash 17.32 17.32
line 13 Long Jump 2.17m 2.17
line 14 Run 02:10.7 130.7
line 15 Dash 11.98 11.98
line 16 Run 02:27.7 147.7
line 17 Dash 59.82 59.82
line 18 Dash 14.11 14.11
line 19 Dash 25.38 25.38
line 20 Dash 12.02 12.02
line 21 Run 02:12.8 132.8
line 22 Run 04:23.9 263.9
line 23 Run 02:20.0 140.0
line 24 Dash 17.18 17.18
line 25 Dash 01:23.7 83.7
line 26 Dash 13.82 13.82
line 27 Dash 15.06 15.06
line 28 Dash 01:03.8 63.8
line 29 Dash 31.98 31.98
line 30 Dash 12.59 12.59
line 31 Dash 01:00.0 60.0
line 32 High Jump 1.25m 1.25
line 33 Dash 13.04 13.04
line 34 Race Walk 29:15.4 1755.4
line 35 Run 02:16.0 136.0
line 36 Run 04:57.0 297.0
line 3

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_92146/2111666615.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  athletes.loc[rowIndex, 'RESULT_CONV'] = result_out


line 1388 400m Hurdles 01:11.2 71.2
line 1389 400m Hurdles 01:08.7 68.7
line 1390 300m Hurdles 56.17 56.17
line 1391 300m Hurdles 54.8 54.8
line 1392 80 Meter Hurdles 15.43 15.43
line 1393 80m Hurdles 19.59 19.59
line 1394 200m Hurdles 45.79 45.79
line 1395 10000m Cross Country 38:18:00 137880.0
line 1396 10000m Cross Country 37:54:00 136440.0
line 1397 10000m Cross Country 40:00:00 144000.0
line 1398 10000m Cross Country 46:17:00 166620.0
line 1399 10000m Cross Country 43:31:00 156660.0
line 1400 10000m Cross Country 39:07:00 140820.0
line 1401 10000m Cross Country 37:43:00 135780.0
line 1402 10000m Cross Country 44:49:00 161340.0
line 1403 6000m Cross Country 25:47:00 92820.0
line 1404 10000m Cross Country 47:29:00 170940.0
line 1405 10000m Cross Country 40:21:00 145260.0
line 1406 400m 54.66 54.66
line 1407 110m Hurdles 14.3 14.3
line 1408 Pole Vault 3.3 3.3
line 1409 Triple Jump 12.1 12.1
line 1410 Long Jump 7.47 7.47
line 1411 Long Jump 7.22 7.22
line 1412 Long Jump 5.72 5.72
line

line 3069 Dash 14.88 14.88
line 3070 Run 02:40.3 160.3
line 3071 Run 02:24.3 144.3
line 3072 Run 02:32.0 152.0
line 3073 Run 02:20.3 140.3
line 3074 Dash 13.74 13.74
line 3075 Long Jump 1.39m 1.39
line 3076 Dash 14.09 14.09
line 3077 Run 02:22.3 142.3
line 3078 Long Jump 1.50m 1.5
line 3079 Dash 13.49 13.49
line 3080 Dash 15.68 15.68
line 3081 Dash 39.2 39.2
line 3082 Dash 18.53 18.53
line 3083 Long Jump 1.57m 1.57
line 3084 Dash 13.4 13.4
line 3085 Shot Put 4.36m 4.36
line 3086 Dash 17.94 17.94
line 3087 Dash 38.09 38.09
line 3088 Dash 13.8 13.8
line 3089 Long Jump 1.81m 1.81
line 3090 Dash 36.56 36.56
line 3091 Dash 16.65 16.65
line 3092 Long Jump 1.50m 1.5
line 3093 Dash 13.28 13.28
line 3094 Dash 13.66 13.66
line 3095 Dash 35.26 35.26
line 3096 Dash 14.15 14.15
line 3097 Dash 12.68 12.68
line 3098 Dash 32.54 32.54
line 3099 Dash 01:13.3 73.3
line 3100 Dash 14.41 14.41
line 3101 Long Jump 3.60m 3.6
line 3102 Dash 12.73 12.73
line 3103 Dash 31.73 31.73
line 3104 Long Jump 2.88m 2.88


line 4508 Dash 14.36 14.36
line 4509 Dash 14.34 14.34
line 4510 Dash 31.31 31.31
line 4511 Dash 14.32 14.32
line 4512 Dash 14.33 14.33
line 4513 Dash 01:07.9 67.9
line 4514 Run 02:49.6 169.6
line 4515 Run 02:54.1 174.1
line 4516 Dash 01:11.7 71.7
line 4517 Dash 14.59 14.59
line 4518 Long Jump 4.01m 4.01
line 4519 Dash 14.42 14.42
line 4520 Dash 13.71 13.71
line 4521 Dash 27.99 27.99
line 4522 Dash 13.44 13.44
line 4523 Dash 58.57 58.57
line 4524 Dash 25.68 25.68
line 4526 Dash 13.11 13.11
line 4527 Dash 12.9 12.9
line 4528 Dash 01:16.0 76.0
line 4529 Run 02:49.3 169.3
line 4530 Dash 01:11.4 71.4
line 4531 Dash 14.42 14.42
line 4532 Dash 14.45 14.45
line 4533 Dash 14.73 14.73
line 4534 Dash 01:12.6 72.6
line 4535 Long Jump 4.30m 4.3
line 4536 Run 02:44.2 164.2
line 4537 Dash 01:11.7 71.7
line 4538 Run 05:42.7 342.7
line 4539 Dash 30.4 30.4
line 4540 Run 08:59.5 539.5
line 4541 Run 08:09.2 489.2
line 4542 Dash 14.57 14.57
line 4543 Dash 16.88 16.88
line 4544 Dash 01:26.0 86.0
line 4545 D

...

line 7436 Relay 48.1 48.1
line 7437 Relay 19.33 19.33
line 7438 Relay 15.97 15.97
line 7439 Relay 54.69 54.69
line 7440 Relay 54.03 54.03
line 7441 Relay 04:24.3 264.3
line 7442 Relay 20.3 20.3
line 7443 Relay 18.73 18.73
line 7444 Relay 15.47 15.47
line 7445 Relay 24.86 24.86
line 7446 Relay 21.86 21.86
line 7447 Relay 03:49.3 229.3
line 7448 Relay 04:29.1 269.1
line 7449 Relay 20.47 20.47
line 7450 Distance Medley 12:54.1 774.1
line 7451 Sprint Medley 02:31.8 151.8
line 7452 Distance Medley 14:32.7 872.7
line 7453 Relay 52.84 52.84
line 7454 Relay 04:33.3 273.3
line 7455 Relay 01:10.3 70.3
line 7456 Relay 01:05.2 65.2
line 7457 Relay 57.76 57.76
line 7458 Relay 01:03.5 63.5
line 7459 Relay 59.38 59.38
line 7460 Relay 01:01.8 61.8
line 7461 Relay 57.73 57.73
line 7462 Relay 14.27 14.27
line 7463 Relay 18.92 18.92
line 7464 Relay 15.98 15.98
line 7465 Relay 14.28 14.28
line 7466 Relay 04:32.7 272.7
line 7467 Relay 05:33.8 333.8
line 7468 Relay 16.15 16.15
line 7469 Relay 16.45 16.45
li

line 8921 Long Jump 4.96m 4.96
line 8922 Long Jump 4.72m 4.72
line 8923 Long Jump 5.23m 5.23
line 8924 Long Jump 5.35m 5.35
line 8925 Long Jump 5.08m 5.08
line 8926 Long Jump 5.70m 5.7
line 8927 Long Jump 4.69m 4.69
line 8928 Long Jump 5.12m 5.12
line 8929 Long Jump 4.85m 4.85
line 8930 Long Jump 4.75m 4.75
line 8931 Long Jump 4.35m 4.35
line 8932 Long Jump 4.56m 4.56
line 8933 Long Jump 4.41m 4.41
line 8934 Long Jump 4.39m 4.39
line 8935 Long Jump 4.49m 4.49
line 8936 Long Jump 5.42m 5.42
line 8937 Long Jump 5.75m 5.75
line 8938 Long Jump 3.37m 3.37
line 8939 Long Jump 5.49m 5.49
line 8940 Long Jump 4.81m 4.81
line 8941 Long Jump 4.78m 4.78
line 8942 Long Jump 4.11m 4.11
line 8943 Long Jump 3.98m 3.98
line 8944 Long Jump 4.57m 4.57
line 8945 Long Jump 4.46m 4.46
line 8946 Long Jump 4.22m 4.22
line 8947 Long Jump 4.17m 4.17
line 8948 Long Jump 5.21m 5.21
line 8949 Long Jump 4.60m 4.6
line 8950 Hurdles 16.28 16.28
line 8951 Hurdles 20.97 20.97
line 8952 Hurdles 18.19 18.19
line 8953 Hur

line 10426 Dash 24.22 24.22
line 10427 Dash 24.17 24.17
line 10428 Dash 25.29 25.29
line 10429 Dash 23.66 23.66
line 10430 Dash 23.79 23.79
line 10431 Dash 24.07 24.07
line 10432 Dash 24.28 24.28
line 10433 Dash 27.73 27.73
line 10434 Dash 24.45 24.45
line 10435 Dash 24.15 24.15
line 10436 Dash 24.94 24.94
line 10437 Dash 24.9 24.9
line 10438 Dash 22.47 22.47
line 10439 Dash 22.8 22.8
line 10440 Dash 22.09 22.09
line 10441 Dash 22.58 22.58
line 10442 Dash 24.1 24.1
line 10443 Dash 22.63 22.63
line 10444 Dash 23.68 23.68
line 10445 Dash 23 23.0
line 10446 Dash 23.36 23.36
line 10447 Dash 32.73 32.73
line 10448 Dash 23.26 23.26
line 10449 Dash 23.45 23.45
line 10450 Dash 23.82 23.82
line 10451 Dash 26.18 26.18
line 10452 Dash 23.19 23.19
line 10453 Dash 23.45 23.45
line 10454 Dash 24.58 24.58
line 10455 Dash 23.87 23.87
line 10456 Dash 23.28 23.28
line 10457 Dash 23.47 23.47
line 10458 Dash 25.01 25.01
line 10459 Dash 24.67 24.67
line 10460 Dash 24.35 24.35
line 10461 Dash 23.94 23.94
li

line 11935 Dash 14.52 14.52
line 11936 Dash 15.21 15.21
line 11937 Dash 15.44 15.44
line 11938 Dash 12.66 12.66
line 11939 Dash 13.18 13.18
line 11940 Dash 13.91 13.91
line 11941 Dash 13.9 13.9
line 11942 Dash 13.76 13.76
line 11943 Dash 14.38 14.38
line 11944 Dash 14.74 14.74
line 11945 Dash 14.03 14.03
line 11946 Dash 15.37 15.37
line 11947 Dash 15.23 15.23
line 11948 Dash 15.46 15.46
line 11949 Dash 14.85 14.85
line 11950 Dash 14.16 14.16
line 11951 Dash 14.03 14.03
line 11952 Dash 15.24 15.24
line 11953 Dash 14.44 14.44
line 11954 Dash 13.89 13.89
line 11955 Dash 13.98 13.98
line 11956 Dash 13.88 13.88
line 11957 Dash 15.19 15.19
line 11958 Dash 13.25 13.25
line 11959 Dash 13.51 13.51
line 11960 Dash 13.74 13.74
line 11961 Dash 15.73 15.73
line 11962 Dash 14.47 14.47
line 11963 Dash 14.85 14.85
line 11964 Dash 16.44 16.44
line 11965 Dash 15.4 15.4
line 11966 Dash 15.01 15.01
line 11967 Dash 12.23 12.23
line 11968 Dash 14.86 14.86
line 11969 Dash 14.97 14.97
line 11970 Dash 14.87 14

line 13424 Relay 46.94 46.94
line 13425 Relay 46.97 46.97
line 13426 Relay 52.12 52.12
line 13427 Relay 51.42 51.42
line 13428 Relay 52.76 52.76
line 13429 Relay 52.95 52.95
line 13430 Relay 46 46.0
line 13431 Relay 45.86 45.86
line 13432 Relay 47.93 47.93
line 13433 Relay 49.98 49.98
line 13434 Relay 49.35 49.35
line 13435 Relay 47.49 47.49
line 13436 Relay 47.05 47.05
line 13437 Relay 52.19 52.19
line 13438 Relay 52.28 52.28
line 13439 Relay 56.89 56.89
line 13440 Relay 52.36 52.36
line 13441 Relay 01:05.1 65.1
line 13442 Relay 53.45 53.45
line 13443 Relay 57.68 57.68
line 13444 Relay 52.35 52.35
line 13445 Relay 44.9 44.9
line 13446 Relay 45.89 45.89
line 13447 Relay 44.47 44.47
line 13448 Relay 45.26 45.26
line 13449 Relay 42.98 42.98
line 13450 Relay 47.26 47.26
line 13451 Relay 43.86 43.86
line 13452 Relay 47.55 47.55
line 13453 Relay 54.36 54.36
line 13454 Relay 01:01.2 61.2
line 13455 Relay 46.97 46.97
line 13456 Relay 50.38 50.38
line 13457 Relay 58.04 58.04
line 13458 Relay 4

In [459]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Taiwan Open/')

athletes.to_csv("athletes.csv", encoding='utf-8')

In [460]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Taiwan Open/')

benchmarks = pd.read_csv("2025 Taiwan Open Benchmarks.csv")


In [461]:
benchmarks

,EVENT,GENDER,BENCHMARK
0,100m,Male,10.68
1,100m,Female,12.2
2,200m,Male,21.8
3,200m,Female,26.14
4,400m,Male,49.56
5,400m,Female,57.48
6,800m,Male,01:58.7
7,800m,Female,02:19.0
8,1500m,Male,04:07.7
9,1500m,Female,05:01.1


In [462]:
for col in benchmarks.columns:
    benchmarks[col] = benchmarks[col].astype(str)
    benchmarks[col] = benchmarks[col].str.replace('\xa0', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('\r', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('\n', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.strip()


In [463]:
#SEAG_filtered=SEAG[SEAG['HEAT'].isnull() & SEAG['SUB_EVENT'].isnull()]

In [464]:
# Convert time for benchmarks and results

for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string = benchmarks.iloc[rowIndex,0]
    
    metric=benchmarks.iloc[rowIndex,2]
    
    if metric==None:
        continue
        
    out = convert_time(i, input_string, metric)
    
    print(rowIndex, input_string, out)
     
    benchmarks.loc[rowIndex, 'Metric'] = out

0 100m 10.68
1 100m 12.2
2 200m 21.8
3 200m 26.14
4 400m 49.56
5 400m 57.48
6 800m 118.7
7 800m 139.0
8 1500m 247.7
9 1500m 301.1
10 5000m 952.4
11 5000m 1185.0
12 3000m Steeplechase(0.914m) 607.0
13 3000m Steeplechase(0.762m) 755.0
14 110m Hurdles(1.067m) 14.5
15 100m Hurdles(0.838m) 14.5
16 400m Hurdles(0.914m) 53.0
17 400m Hurdles(0.762m) 68.04
18 High Jump 2.09
19 High Jump 1.68
20 Long Jump 7.3
21 Long Jump 5.7
22 Triple Jump 14.7
23 Triple Jump 11.65
24 Shot Put(7.26kg) 14.38
25 Shot Put(4kg) 12.3
26 Discus Throw(2kg) 43.5
27 Discus Throw(1kg) 38.5
28 Javelin Throw(800g) 64.0
29 Javelin Throw(600g) 45.15
30 Hammer Throw(7.26kg) 50.0
31 Hammer Throw(4kg) 50.0
32 Pole Vault 4.8
33 Pole Vault 3.3


In [465]:
benchmarks

,EVENT,GENDER,BENCHMARK,Metric
0,100m,Male,10.68,10.68
1,100m,Female,12.2,12.20
2,200m,Male,21.8,21.80
3,200m,Female,26.14,26.14
4,400m,Male,49.56,49.56
5,400m,Female,57.48,57.48
6,800m,Male,01:58.7,118.70
7,800m,Female,02:19.0,139.00
8,1500m,Male,04:07.7,247.70
9,1500m,Female,05:01.1,301.10


In [466]:
mask = benchmarks['EVENT'].str.contains(r'jump|throw|Pole|put|Jump|Throw|pole|Put', na=True)

benchmarks.loc[mask, '2%']=benchmarks['Metric']*0.98
benchmarks.loc[~mask, '2%']=benchmarks['Metric']*1.02
benchmarks.loc[mask, '3.5%']=benchmarks['Metric']*0.975
benchmarks.loc[~mask, '3.5%']=benchmarks['Metric']*1.035
benchmarks.loc[mask, '5%']=benchmarks['Metric']*0.95
benchmarks.loc[~mask, '5%']=benchmarks['Metric']*1.05


#benchmarks.iloc[5, [1]]='10000m run'
#benchmarks.iloc[28, [1]]='10000m run'
#benchmarks.iloc[26, [1]]='1500m'


In [467]:
benchmarks

,EVENT,GENDER,BENCHMARK,Metric,2%,3.5%,5%
0,100m,Male,10.68,10.68,10.8936,11.05380,11.2140
1,100m,Female,12.2,12.20,12.4440,12.62700,12.8100
2,200m,Male,21.8,21.80,22.2360,22.56300,22.8900
3,200m,Female,26.14,26.14,26.6628,27.05490,27.4470
4,400m,Male,49.56,49.56,50.5512,51.29460,52.0380
5,400m,Female,57.48,57.48,58.6296,59.49180,60.3540
6,800m,Male,01:58.7,118.70,121.0740,122.85450,124.6350
7,800m,Female,02:19.0,139.00,141.7800,143.86500,145.9500
8,1500m,Male,04:07.7,247.70,252.6540,256.36950,260.0850
9,1500m,Female,05:01.1,301.10,307.1220,311.63850,316.1550


In [468]:
'''
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'110m Hurdles(0.914m)', value='110m Hurdles')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'110m Hurdles(0.762m)', value='110m Hurdles')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'400m Hurdles(0.840m)', value='400m Hurdles')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'400m Hurdles(0.762m)', value='400m Hurdles')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Javelin Throw(700g)', value='Javelin Throw')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Javelin Throw(500g)', value='Javelin Throw')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Hammer Throw(5kg)', value='Hammer Throw')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Hammer Throw(3kg)', value='Hammer Throw')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Shot Put(5kg)', value='Shot Put')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Shot Put(3kg)', value='Shot Put')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Discus Throw(1.5kg)', value='Discus Throw')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Discus Throw(1kg)', value='Discus Throw')
'''

"\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'110m Hurdles(0.914m)', value='110m Hurdles')\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'110m Hurdles(0.762m)', value='110m Hurdles')\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'400m Hurdles(0.840m)', value='400m Hurdles')\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'400m Hurdles(0.762m)', value='400m Hurdles')\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Javelin Throw(700g)', value='Javelin Throw')\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Javelin Throw(500g)', value='Javelin Throw')\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Hammer Throw(5kg)', value='Hammer Throw')\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Hammer Throw(3kg)', value='Hammer Throw')\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Shot Put(5kg)', value='Shot Put')\nbenchmarks['MAP

In [469]:
# There is a problem with RESULTS column being changed after this statement

#df = athletes.reset_index().merge(benchmarks.reset_index(), on=['MAPPED_EVENT','GENDER'], how='left')
#df = athletes.merge(benchmarks, on=['EVENT','GENDER'], how='left')


In [470]:
# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

df = pd.merge(
    left=athletes, 
    right=benchmarks,
    how='left',
    left_on=['MAPPED_EVENT', 'GENDER'],
    right_on=['EVENT', 'GENDER'],
)

In [471]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,year_extract,age_extract,MAPPED_EVENT,RESULT_CONV,EVENT_y,BENCHMARK,Metric,2%,3.5%,5%
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34.0,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,1990.0,35.0,Hammer Throw(7.26kg),32.4,Hammer Throw(7.26kg),50,50.0,49.000,48.75,47.50
1,"Koh, Wei Shien",4.41m,NUS,29.0,3,Open,Pole Vault,0,,L435B06,...,1995.0,30.0,Pole Vault,4.41,Pole Vault,4.8,4.8,4.704,4.68,4.56
2,"Wen Qiang, Koh",07:38.0,SIT,27.0,4,Tertiary Men,Run,2400,,,...,1997.0,28.0,,458.0,NaN,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4.0,7,5-Jun,Run,60,,nan,...,2019.0,6.0,,15.03,NaN,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50.0,5,Masters,Javelin Throw,0,,nan,...,2074.0,51.0,,22.43,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14866,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23.0,2,Open,Relay,400,Trial,None,...,2002.0,23.0,4 x 100m,42.06,NaN,NaN,NaN,NaN,NaN,NaN
14867,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16.0,2,Open,Relay,400,Trial,None,...,2009.0,16.0,4 x 100m,42.06,NaN,NaN,NaN,NaN,NaN,NaN
14868,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15.0,2,Open,Relay,400,Trial,None,...,2010.0,15.0,4 x 100m,42.06,NaN,NaN,NaN,NaN,NaN,NaN
14869,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16.0,2,Open,Relay,400,Trial,None,...,2009.0,16.0,4 x 100m,42.06,NaN,NaN,NaN,NaN,NaN,NaN


In [472]:
df[df['MAPPED_EVENT']=='5000m']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,year_extract,age_extract,MAPPED_EVENT,RESULT_CONV,EVENT_y,BENCHMARK,Metric,2%,3.5%,5%
37,"Kato, Takao",19:16.2,UNA,51.0,1,Masters,Run,5000,,nan,...,2073.0,52.0,5000m,1156.2,5000m,15:52.4,952.4,971.448,985.734,1000.02
69,SHOHIB MARICAN,17:56.6,MR25,27.0,14,None,5000m,None,None,None,...,1997.0,28.0,5000m,1076.6,5000m,15:52.4,952.4,971.448,985.734,1000.02
90,"Mevada, Nirmaben Jagdish Bha",35:26.2,INDIA,35.0,1,Masters,Run,5000,,nan,...,1989.0,36.0,5000m,2126.2,5000m,19:45.0,1185.0,1208.700,1226.475,1244.25
102,"Kanojiya, Priya Alpesh Kumar",28:03.4,INDIA,40.0,1,Masters,Run,5000,,nan,...,1984.0,41.0,5000m,1683.4,5000m,19:45.0,1185.0,1208.700,1226.475,1244.25
104,"Saha, Sayantani",30:00.0,INDIA,43.0,2,Masters,Run,5000,,nan,...,1981.0,44.0,5000m,1800.0,5000m,19:45.0,1185.0,1208.700,1226.475,1244.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13991,"VENKATRAMAN, SUBBHIKSHAN",20:21.3,Millennia Institute,19.0,25,Open,Run,5000,None,S522C06,...,2006.0,19.0,5000m,1221.3,5000m,15:52.4,952.4,971.448,985.734,1000.02
13992,"Koh, Royston",20:10.4,Lacticbuds,18.0,22,Open,Run,5000,None,R484G07,...,2007.0,18.0,5000m,1210.4,5000m,15:52.4,952.4,971.448,985.734,1000.02
14701,"NG, YEW CHEO",24:16.9,Cougars Athletic Association,39.0,3,Open,Run,5000,None,Y971Z86,...,1986.0,39.0,5000m,1456.9,5000m,19:45.0,1185.0,1208.700,1226.475,1244.25
14702,"Ng, Sherlene",20:59.9,TeamFabian,24.0,2,Open,Run,5000,None,S638D01,...,2001.0,24.0,5000m,1259.9,5000m,19:45.0,1185.0,1208.700,1226.475,1244.25


In [473]:
# Change to numeric

df['RESULT_CONV'] = df['RESULT_CONV'].replace(regex=r'-', value=np.NaN)
df['RESULT_CONV'] = df['RESULT_CONV'].replace(regex=r'-', value=np.NaN)
df['RESULT_CONV'] = df['RESULT_CONV'].replace(regex=r'nan', value=np.NaN)

df['age_extract'] = df['age_extract'].replace(regex=r'-', value=np.NaN)
df['age_extract'] = df['age_extract'].replace(regex=r'-', value=np.NaN)
df['age_extract'] = df['age_extract'].replace(regex=r'nan', value=np.NaN)
df['age_extract'] = df['age_extract'].replace(regex=r'None', value=np.NaN)


df[['2%', '3.5%', '5%', 'RESULT_CONV']] = df[['2%', '3.5%', '5%', 'RESULT_CONV']].apply(pd.to_numeric)

df.to_csv("testing.csv", encoding='utf-8')


In [474]:
mask = df['CATEGORY_EVENT'].str.contains(r'Jump|Throw|jump|throw', na=True)

df.loc[mask, 'Delta2'] = df['RESULT_CONV']-df['2%']
df.loc[mask, 'Delta3.5'] = df['RESULT_CONV']-df['3.5%']
df.loc[mask, 'Delta5'] = df['RESULT_CONV']-df['5%']

df.loc[mask, 'Delta_Benchmark'] = df['RESULT_CONV']-df['Metric']

df.loc[~mask, 'Delta2'] =  df['2%'] - df['RESULT_CONV']
df.loc[~mask, 'Delta3.5'] =  df['3.5%'] - df['RESULT_CONV']
df.loc[~mask, 'Delta5'] =  df['5%'] - df['RESULT_CONV']

df.loc[~mask, 'Delta_Benchmark'] = df['Metric'] - df['RESULT_CONV']

#rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']


In [475]:
# Performance metric to filter out athletes

df['PERF_SCALAR']=df['Delta_Benchmark']/df['Metric']*100

In [476]:
#df['TIER'] = np.where((df['Delta_Benchmark']>=0), 'Tier 1',    
#            np.where(((df['Delta_Benchmark']<0) & (df['Delta2']>=0)), 'Tier2', ' '))

df['TIER'] = np.where((df['Delta_Benchmark']>=0), 'Tier 1',    
                                np.where(((df['Delta_Benchmark']<0) & (df['Delta2']>=0)), 'Tier 2',
                                np.where(((df['Delta2']<0) & (df['Delta3.5']>=0)), 'Tier 3', 
                                np.where(((df['Delta3.5']<0) & (df['Delta5']>=0)), 'Tier 4', ' '))))


In [477]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Taiwan Open/')

df_filtered=df[df['PERF_SCALAR']!=' ']

df_filtered.to_csv('df_mapped.csv', sep=',', encoding='utf-8-sig', index=False)


In [478]:
#df[['age_extract']] = df[['age_extract']].apply(pd.to_numeric)

#athletes_selected = df[((df['age_extract']>=15) & (df['age_extract']<=17))]


In [479]:
athletes_selected=df

In [480]:
# Read name variations from GCS name lists bucket (Still in beta)


athletes_selected['NAME'] = athletes_selected['NAME'].str.replace('\xa0', '', regex=True)
athletes_selected['NAME'] = athletes_selected['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
athletes_selected['NAME'] = athletes_selected['NAME'].str.replace('\r', '', regex=True)
athletes_selected['NAME'] = athletes_selected['NAME'].str.replace('\n', '', regex=True)
athletes_selected['NAME'] = df['NAME'].str.strip()


# Read csv from GCS bucket

file_path = "gs://name_variations/name_variations.csv"
names = pd.read_csv(file_path,
                 sep=",",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})

# Iterate over dataframe and replace names

for index, row in names.iterrows():
        
    athletes_selected['NAME'] = athletes_selected['NAME'].replace(regex=rf"^{row['VARIATION']}$", value=f"{row['NAME']}")


In [481]:
# Choose the best result for each event participated by every athlete

best_selection = athletes_selected.sort_values(['MAPPED_EVENT', 'NAME','PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)

athletes=best_selection

In [482]:
# Exclude foreigners from MALAYSIA, THAILAND etc.

final_taiwan_selection = athletes[(athletes['TEAM']!='Malaysia')&(athletes['TEAM']!='THAILAND')&(athletes['TEAM']!='China') 
                       &(athletes['TEAM']!='South Korea')&(athletes['TEAM']!='Laos') 
                       &(athletes['TEAM']!='Philippines')&(athletes['TEAM']!='Piboonbumpen Thailand') 
                       &(athletes['TEAM']!='Chinese Taipei')&(athletes['TEAM']!='Gurkha Contingent') 
                       &(athletes['TEAM']!='Australia')&(athletes['TEAM']!='Piboonbumpen Thailand') 
                       &(athletes['TEAM']!='Hong Kong')&(athletes['TEAM']!='PERAK')&(athletes['TEAM']!='Sri Lanka') 
                       &(athletes['TEAM']!='Indonesia')&(athletes['TEAM']!='THAILAND')&(athletes['TEAM']!='MALAYSIA') 
                       &(athletes['TEAM']!='PHILIPPINES') & (athletes['TEAM']!='SOUTH KOREA')&(athletes['TEAM']!='Waseda') 
                       &(athletes['TEAM']!='LAOS')&(athletes['TEAM']!='CHINESE TAIPEI')
                       &(athletes['TEAM']!='INDIA')&(athletes['TEAM']!='Hong Kong, China')&(athletes['TEAM']!='AIC JAPAN')&(athletes['NATIONALITY']!='GBR')&(athletes['NATIONALITY']!='IND')] 


In [483]:
all_ranking = final_taiwan_selection.sort_values(['MAPPED_EVENT','GENDER','PERF_SCALAR'], ascending=[False, False, False])
all_ranking['Rank'] = all_ranking.groupby(['GENDER', 'MAPPED_EVENT', 'TIER']).cumcount() + 1


In [485]:
all_ranking=all_ranking[(all_ranking['TIER']!=' ')]

In [486]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Taiwan Open/')


all_ranking.to_csv('taiwan_open.csv', sep=',', encoding='utf-8-sig', index=False)


# Clean manual load data

In [133]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/SEA Games/')


df_SHA = pd.read_csv("SEA_manual_additions.csv", encoding='latin-1')

In [134]:
df_SHA

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,NaN,NaN,NaN,Agus Prayogo,1.0,NaN,NaN,NaN,2:32:59,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
1,NaN,NaN,NaN,ÃArlan Estobo Arbois,2.0,NaN,NaN,NaN,2:33:27,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
2,NaN,NaN,NaN,Nguyen Thanh Hoang,3.0,NaN,NaN,NaN,2:35:49,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
3,NaN,NaN,NaN,Tan Huong Leong,4.0,NaN,NaN,NaN,2:40:26,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
4,NaN,NaN,NaN,Vanh Pheara,5.0,NaN,NaN,NaN,2:41:26,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
5,NaN,NaN,NaN,Quoc Luong Trinh,6.0,NaN,NaN,NaN,2:41:36,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
6,NaN,NaN,NaN,Sanchai Namkhet,7.0,NaN,NaN,NaN,2:43:47,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
7,NaN,NaN,NaN,Joanito Fernandes,8.0,NaN,NaN,NaN,2:52:41,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
8,NaN,NaN,NaN,Tony Ah-Thit Payne,NaN,NaN,NaN,NaN,DNF,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
9,NaN,NaN,NaN,Yang Piseth,NaN,NaN,NaN,NaN,DNF,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN


In [135]:
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\xa0', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\r', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\n', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.strip()


In [136]:
df_SHA.to_csv('SEA_manual_additions.csv', index=False, encoding='utf-8')

# Read a csv of name list variations

In [137]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')

names = pd.read_csv("name_variations.csv")

In [138]:
names

,VARIATION,NAME
0,Harry\sIrfan\sCurra,HARRY IRFAN CURRAN
1,"Curran, Harry Irfan",HARRY IRFAN CURRAN
2,Harry Irfan Curran,HARRY IRFAN CURRAN
3,"Curran, Harry Irfann",HARRY IRFAN CURRAN
4,"Curran,\sHarry",HARRY IRFAN CURRAN
...,...,...
370,Goh Xu'en Hannah,"GOH, XU'EN HANNAH"
371,"Low, Jonathan","Low Jun Jie, Jonathan"
372,"Low, Jun Jie Jonathon","Low Jun Jie, Jonathan"
373,"Low, Jun Jie Jonathan","Low Jun Jie, Jonathan"


In [139]:
lines = [line.strip() for line in open('file.csv')]
for x in lines:
    match=re.search(r'^_.*_$',x)
    if match: print x

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (2082507092.py, line 4)

In [ ]:
df['NAME'] = df['NAME'].replace(regex=r'^Harry\sIrfan\sCurra$', value='HARRY IRFAN CURRAN')


In [ ]:
for index, row in names.iterrows():
    print(row['VARIATION'])
    